# 1) Titolo e obiettivi

Lezione 31: TF-IDF e Text Mining - Pesare le parole per importanza

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, perché TF-IDF | 5 min |
| 2 | Teoria: TF, IDF, formula, normalizzazione | 15 min |
| 3 | Schema mentale: workflow TF-IDF | 5 min |
| 4 | Demo: calcolo manuale, cosine similarity, classificazione | 30 min |
| 5 | Esercizi guidati + keyword extraction | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Calcolare **TF-IDF manualmente** | Sai applicare la formula? |
| 2 | Usare **TfidfVectorizer** di sklearn | Sai settare parametri? |
| 3 | Misurare **similarità coseno** | Sai confrontare documenti? |
| 4 | Confrontare **BoW vs TF-IDF** in classificazione | Sai interpretare F1? |
| 5 | Estrarre **keyword** con coefficienti | Sai leggere coef_ di LogReg? |

---

## L'idea centrale: perché TF-IDF

```
PROBLEMA CON BOW:                    SOLUZIONE TF-IDF:

"the" appare 50 volte                "the" ha IDF basso
→ peso 50 (troppo alto!)             → peso finale basso

"algorithm" appare 2 volte           "algorithm" ha IDF alto  
→ peso 2 (troppo basso!)             → peso finale alto

TF-IDF = TF × IDF
       = frequenza locale × rarità globale
```

**Intuizione:** parole rare ma presenti = importanti!

---

## La formula TF-IDF

```
TF(t, d) = numero occorrenze di t in documento d
           ─────────────────────────────────────
           totale parole in d (opzionale)

IDF(t) = log( N / (1 + df(t)) )
         dove N = numero documenti, df(t) = documenti con t

TF-IDF(t, d) = TF(t, d) × IDF(t)

Normalizzazione L2: divide ogni vettore per la sua norma
→ tutti i documenti hanno "lunghezza" 1
→ confronto equo tra doc lunghi e corti
```

---

## Similarità coseno: confrontare documenti

```
         doc A                    doc B
           ↗                        ↗
          /                        /
         / θ (angolo)             /
        ╱─────────────────────────╱
       origine

cosine(A, B) = (A · B) / (||A|| × ||B||)

Valori:
- 1.0 = identici (angolo = 0°)
- 0.0 = ortogonali (angolo = 90°)
- -1.0 = opposti (raro in NLP)
```

**Uso tipico:** ranking documenti simili, ricerca, deduplicazione.

---

## BoW vs TF-IDF: quando usare cosa

| Aspetto | BoW | TF-IDF |
|---------|-----|--------|
| **Formula** | Conteggi raw | Conteggi × IDF |
| **Parole comuni** | Peso alto | Peso basso |
| **Parole rare** | Peso basso | Peso alto |
| **Classificazione** | OK su testi corti | Meglio su testi lunghi |
| **Velocità** | Leggermente più veloce | Simile |
| **Interpretabilità** | Diretta (conteggio) | Indiretta (peso) |

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| Bag of Words | Lezione 30 | Sai usare CountVectorizer? |
| Matrici sparse | scipy | Sai cos'è CSR? |
| Classificazione testo | Lezione 30 | Sai usare MultinomialNB? |
| Regressione logistica | Lezioni 5-6 | Sai leggere coef_? |

**Cosa useremo:** CountVectorizer, TfidfVectorizer, cosine_similarity, MultinomialNB, LogisticRegression.

# 2) Teoria concettuale
- TF (term frequency): quante volte appare una parola in un documento.
- IDF (inverse document frequency): quanto la parola e' rara nel corpus; piu' rara = peso maggiore.
- TF-IDF = TF * IDF con normalizzazione L2 per confrontare documenti di lunghezza diversa.
- Similarita' coseno: misura di vicinanza tra vettori TF-IDF (0=ortogonali, 1=identici).


## Quando usare TF-IDF vs BoW
- TF-IDF: preferibile per classificazione/testo lungo per attenuare parole frequenti poco informative.
- BoW: semplice e veloce su testi brevi/lessico limitato.
- Entrambi richiedono vocabolario fissato sul training e `transform` sui nuovi dati.


# 3) Schema mentale / mappa decisionale
1. Pulisci e tokenizza testi (minuscolo, punteggiatura, stopword).
2. Scelta rappresentazione: BoW se vocab piccolo, TF-IDF se vuoi pesi informativi.
3. Calcola similarita' coseno per confronti/ricerca.
4. Per classificazione: split train/test, prova BoW e TF-IDF, confronta metriche.
5. Per interpretazione: usa pesi/coef per estrarre keyword per classe.


# 4) Sezione dimostrativa
- Demo 1: calcolo manuale TF-IDF e verifica con sklearn.
- Demo 2: similarita' coseno tra documenti.
- Demo 3: confronto BoW vs TF-IDF in classificazione testi.
- Demo 4: parole piu' importanti per classe con Logistic Regression.


## Demo 1 - Calcolo manuale TF-IDF
Perche': capire le formule prima di usare il vectorizer. Checkpoint: stesse forme e valori simili a sklearn.


## Demo 2 - Similarita' coseno
Perche': misurare vicinanza tra documenti nello spazio TF-IDF.


## Demo 3 e 4 - Classificazione e interpretazione
Perche': confrontare BoW vs TF-IDF su sentiment e leggere le parole piu' influenti.


In [ ]:
# Setup
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

corpus = [
    "il gatto mangia il pesce",
    "il cane mangia la carne",
    "il gatto dorme sul divano",
    "il cane gioca nel parco"
]


In [ ]:
# Demo 1: calcolo manuale TF-IDF - vocabolario e token
print("Manuale TF-IDF: vocabolario e token")
all_words = set()
tokenized_docs = []
for doc in corpus:
    tokens = doc.split()
    tokenized_docs.append(tokens)
    all_words.update(tokens)
vocab = sorted(all_words)
print(f"Vocab: {vocab}")
N = len(corpus)
V = len(vocab)
assert V > 0


In [ ]:
# Demo 1: matrice TF (conteggi)
word_to_idx = {w:i for i,w in enumerate(vocab)}
tf_matrix = np.zeros((N, V))
for d_i, tokens in enumerate(tokenized_docs):
    for t in tokens:
        tf_matrix[d_i, word_to_idx[t]] += 1
print("TF grezzi:", pd.DataFrame(tf_matrix, columns=vocab))
assert tf_matrix.shape == (N, V)


In [ ]:
# Demo 1: IDF
# df = in quanti documenti appare il termine
df_counts = np.sum(tf_matrix > 0, axis=0)
idf_values = np.log((N + 1) / (df_counts + 1)) + 1
idf_df = pd.DataFrame({'parola': vocab, 'df': df_counts, 'idf': idf_values})
print(idf_df)
assert idf_values.shape[0] == V


In [ ]:
# Demo 1: TF-IDF e normalizzazione L2
# TF-IDF non normalizzato
tfidf_matrix = tf_matrix * idf_values
# normalizzazione riga per riga
norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)
tfidf_norm = tfidf_matrix / norms
print(pd.DataFrame(tfidf_norm.round(3), columns=vocab))
assert tfidf_norm.shape == (N, V)


In [ ]:
# Demo 1: verifica con sklearn
vec_tfidf = TfidfVectorizer()
X_skl = vec_tfidf.fit_transform(corpus)
vocab_skl = vec_tfidf.get_feature_names_out()
df_skl = pd.DataFrame(X_skl.toarray().round(3), columns=vocab_skl)
print(df_skl)
assert df_skl.shape[1] == len(vocab_skl)


In [ ]:
# Demo 2: similarita' coseno
sim_matrix = cosine_similarity(X_skl)
df_sim = pd.DataFrame(sim_matrix.round(3), index=[f"D{i+1}" for i in range(N)], columns=[f"D{i+1}" for i in range(N)])
print(df_sim)
assert df_sim.shape == (N, N)


In [ ]:
# Demo 3: dataset di sentiment per confronto BoW vs TF-IDF
recensioni = [
    ("Prodotto eccezionale, lo consiglio a tutti", 1),
    ("Ottimo acquisto, sono molto soddisfatto", 1),
    ("Qualita superiore, prezzo giusto", 1),
    ("Spedizione veloce, prodotto perfetto", 1),
    ("Fantastico, supera le aspettative", 1),
    ("Pessimo, arrivato rotto", 0),
    ("Non funziona, soldi sprecati", 0),
    ("Esperienza terribile, non comprare", 0),
    ("Prodotto difettoso e assistenza assente", 0),
    ("Deluso, qualita bassa", 0)
]
texts, labels = zip(*recensioni)
X_train_txt, X_test_txt, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42, stratify=labels)


In [ ]:
# Demo 3: BoW vs TF-IDF con due classificatori
bow_vec = CountVectorizer()
tfidf_vec = TfidfVectorizer()

X_train_bow = bow_vec.fit_transform(X_train_txt)
X_test_bow = bow_vec.transform(X_test_txt)
X_train_tfidf = tfidf_vec.fit_transform(X_train_txt)
X_test_tfidf = tfidf_vec.transform(X_test_txt)

classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=500, random_state=42)
}

rows = []
for rep_name, (X_tr, X_te) in {'BoW': (X_train_bow, X_test_bow), 'TF-IDF': (X_train_tfidf, X_test_tfidf)}.items():
    for clf_name, clf in classifiers.items():
        clf.fit(X_tr, y_train)
        acc = accuracy_score(y_test, clf.predict(X_te))
        rows.append({'rappresentazione': rep_name, 'modello': clf_name, 'accuracy': acc})
res_cls = pd.DataFrame(rows)
print(res_cls)
assert not res_cls.empty


In [ ]:
# Demo 4: parole piu' importanti per classe (Logistic Regression su TF-IDF)
X_full = tfidf_vec.fit_transform(texts)
clf_full = LogisticRegression(max_iter=1000, random_state=42)
clf_full.fit(X_full, labels)
feature_names = tfidf_vec.get_feature_names_out()
coef = clf_full.coef_[0]

def top_terms(coefs, feature_names, top=5):
    idx_sorted = np.argsort(coefs)
    neg = [(feature_names[i], coefs[i]) for i in idx_sorted[:top]]
    pos = [(feature_names[i], coefs[i]) for i in idx_sorted[-top:][::-1]]
    return neg, pos

neg_terms, pos_terms = top_terms(coef, feature_names, top=5)
print("Top parole classe negativa:", neg_terms)
print("Top parole classe positiva:", pos_terms)


# 5) Esercizi svolti (passo-passo)
## Esercizio 31.1 - Calcolo TF-IDF step-by-step
Obiettivo: replicare il calcolo TF-IDF su un mini-corpus per verificare formule e normalizzazione.


In [ ]:
# Soluzione esercizio 31.1
mini_corpus = [
    "machine learning is great",
    "deep learning is a subset of machine learning",
    "neural networks power deep learning"
]
vec = TfidfVectorizer()
X_mini = vec.fit_transform(mini_corpus)
print(pd.DataFrame(X_mini.toarray().round(3), columns=vec.get_feature_names_out()))
assert X_mini.shape[0] == len(mini_corpus)


## Esercizio 31.2 - Mini motore di ricerca
Obiettivo: indicizzare documenti con TF-IDF e rispondere a una query ordinando per similarita'.


In [ ]:
# Soluzione esercizio 31.2
corpus_docs = [
    "Introduzione al machine learning e ai modelli di classificazione",
    "Deep learning con reti neurali e tecniche di ottimizzazione",
    "Metodi di clustering e riduzione dimensionale",
    "Tecniche di text mining e NLP"
]
vec_search = TfidfVectorizer()
X_docs = vec_search.fit_transform(corpus_docs)

query = "reti neurali per deep learning"
q_vec = vec_search.transform([query])
scores = cosine_similarity(q_vec, X_docs).flatten()
order = scores.argsort()[::-1]
print("Ranking risultati:")
for idx in order:
    print(f"Doc {idx+1} score {scores[idx]:.3f}: {corpus_docs[idx]}")
assert scores.shape[0] == len(corpus_docs)


## Esercizio 31.3 - Estrazione keywords
Obiettivo: usare TF-IDF per estrarre le parole piu' pesanti da un documento rispetto a un corpus di riferimento.


Esecuzione estrazione keywords con TF-IDF.


In [ ]:
# Soluzione esercizio 31.3
vec_kw = TfidfVectorizer()
vec_kw.fit(corpus_docs)

def extract_keywords(document: str, vectorizer, top_n: int = 5) -> list:
    vec = vectorizer.transform([document])
    scores = vec.toarray().flatten()
    idx = scores.argsort()[::-1][:top_n]
    terms = vectorizer.get_feature_names_out()
    return [(terms[i], scores[i]) for i in idx if scores[i] > 0]

sample_doc = corpus_docs[0]
print(extract_keywords(sample_doc, vec_kw, top_n=5))


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **TF-IDF pesa per rarità** | Parole comuni → peso basso |
| 2 | **Cosine similarity per confronti** | Indipendente dalla lunghezza |
| 3 | **Normalizzazione L2 inclusa** | TfidfVectorizer la applica automaticamente |
| 4 | **Confronta sempre BoW vs TF-IDF** | Non assumere che uno sia sempre meglio |
| 5 | **coef_ per keyword extraction** | LogReg interpreta quali parole contano |

---

## Confronto sintetico: parametri TfidfVectorizer

| Parametro | Default | Effetto | Quando cambiare |
|-----------|---------|---------|-----------------|
| `ngram_range` | (1,1) | Solo unigrammi | (1,2) per bigrammi |
| `max_features` | None | Tutti i termini | 5000-10000 per limitare |
| `min_df` | 1 | Anche termini rari | 2-5 per rimuovere rari |
| `max_df` | 1.0 | Anche troppo comuni | 0.9 per rimuovere "the" |
| `norm` | 'l2' | Normalizza vettori | None per valori raw |
| `sublinear_tf` | False | TF = 1 + log(tf) | True per corpus grandi |

---

## Perché questi concetti funzionano

### 1) IDF come filtro informativo

```
Parola      DF (doc con parola)    IDF = log(N/DF)
─────────   ─────────────────      ───────────────
"the"       1000/1000              log(1) = 0 (inutile!)
"algorithm" 10/1000                log(100) = 2 (informativa!)
"quantum"   1/1000                 log(1000) = 3 (molto rara!)
```

### 2) Cosine vs distanza euclidea

```
Doc A: [1, 0, 0, 5]  (50 parole)
Doc B: [2, 0, 0, 10] (100 parole)

Euclidea: ||A - B|| = grande (sembrano diversi!)
Cosine:   cos(A, B) = 1.0 (sono IDENTICI in direzione!)

TF-IDF + cosine = invariante alla lunghezza
```

---

## Reference card metodi

| Metodo | Input | Output | Note |
|--------|-------|--------|------|
| `TfidfVectorizer()` | - | vectorizer | Configura |
| `.fit_transform(docs)` | lista stringhe | sparse matrix | TF-IDF |
| `.transform(docs)` | lista stringhe | sparse matrix | Nuovo testo |
| `cosine_similarity(X, Y)` | matrici | similarity matrix | 0-1 |
| `LogisticRegression().coef_` | - | array pesi | Per interpretazione |
| `.get_feature_names_out()` | - | array termini | Vocabolario |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| Refit su test | Vocabolario/IDF diversi | Solo transform() |
| Cosine su BoW non normalizzato | Lunghezza influisce | Usa TF-IDF o normalizza |
| Ignorare stopwords | "the" domina | stop_words='english' |
| sublinear_tf non usato | TF troppo alto su parole ripetute | sublinear_tf=True |
| Confrontare doc con vocabolario diverso | Dimensioni non matchano | Stesso vectorizer |

---

## Template TF-IDF + classificazione

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# 1) Vettorizza
vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, min_df=2)
X = vec.fit_transform(train_docs)
X_test = vec.transform(test_docs)

# 2) Classifica
clf = LogisticRegression(max_iter=1000)
clf.fit(X, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

# 3) Estrai keyword per classe
feature_names = vec.get_feature_names_out()
for i, class_name in enumerate(clf.classes_):
    top_idx = clf.coef_[i].argsort()[-10:][::-1]
    keywords = [feature_names[j] for j in top_idx]
    print(f"{class_name}: {keywords}")
```

---

## Prossimi passi

| Lezione | Argomento | Collegamento |
|---------|-----------|--------------|
| 32 | Sentiment Analysis | Polarità positiva/negativa |
| 33 | Named Entity Recognition | Estrazione entità |
| 34+ | Document Intelligence | Struttura documenti |

# 7) Checklist di fine lezione
- [ ] Ho calcolato TF-IDF e confrontato i risultati con sklearn.
- [ ] Ho usato cosine similarity per verificare vicinanza tra documenti.
- [ ] Ho confrontato BoW e TF-IDF in un task di classificazione.
- [ ] Ho estratto keyword o coefficienti per interpretare i modelli.
- [ ] Ho mantenuto lo stesso vectorizer per train/test e query.

Glossario
- TF: term frequency, conteggio nel documento.
- IDF: inverse document frequency, peso inverso alla frequenza nei documenti.
- TF-IDF: prodotto TF e IDF, normalizzato.
- Cosine similarity: misura di angolo tra vettori.
- BoW: rappresentazione a conteggi.
- Vocabolario: insieme di termini indicizzati.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-02-05 | Creazione: TF-IDF base e sklearn |
| 1.1 | 2024-02-12 | Aggiunta cosine similarity |
| 2.0 | 2024-02-18 | Integrato confronto BoW vs TF-IDF |
| 2.1 | 2024-02-22 | Refactor con keyword extraction |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII formula TF-IDF, intuizione IDF come filtro, cosine vs euclidea diagram, BoW vs TF-IDF comparison table, 5 take-home messages, tabella parametri TfidfVectorizer, template completo classificazione + keyword, reference card |

---

## Note per lo studente

TF-IDF è la rappresentazione standard per NLP classico:

| Tecnica | Uso principale |
|---------|----------------|
| BoW | Baseline, testi corti |
| TF-IDF | Classificazione, ricerca |
| Word embeddings | Semantica, deep learning |

**Workflow NLP consolidato:**
1. Pulizia → 2. TF-IDF → 3. Modello → 4. Interpretazione

**Prossima tappa:** Lesson 32 - Sentiment Analysis (classificazione polarità)